In [1]:
import sys
sys.path.append("..")
from ctdi_treatment.env_setup_start import *
import time
license_keys = set_envvars('/home/ubuntu/hasham/jsl_keys.json')
spark = start_sparknlp(license_keys)

Desired SparkNLP Version: 3.1.3
Desired SparkNLP-JSL Version: 3.1.3
Real Spark NLP Version : 3.1.1
Real Spark NLP_JSL Version : 3.1.3


In [2]:
pair_nd = ["strength","duration","frequency","dosage","route","form",
                           "relativedate","administration","cyclelength"]
pair_dr = ['drug', 'treatment']
pairs = [ f'{j}-{i}' for i in pair_nd for j in pair_dr]
pairs += [ i.split('-')[1]+'-'+i.split('-')[0] for i in pairs ]
print (len(pairs))
pairs

36


['drug-strength',
 'treatment-strength',
 'drug-duration',
 'treatment-duration',
 'drug-frequency',
 'treatment-frequency',
 'drug-dosage',
 'treatment-dosage',
 'drug-route',
 'treatment-route',
 'drug-form',
 'treatment-form',
 'drug-relativedate',
 'treatment-relativedate',
 'drug-administration',
 'treatment-administration',
 'drug-cyclelength',
 'treatment-cyclelength',
 'strength-drug',
 'strength-treatment',
 'duration-drug',
 'duration-treatment',
 'frequency-drug',
 'frequency-treatment',
 'dosage-drug',
 'dosage-treatment',
 'route-drug',
 'route-treatment',
 'form-drug',
 'form-treatment',
 'relativedate-drug',
 'relativedate-treatment',
 'administration-drug',
 'administration-treatment',
 'cyclelength-drug',
 'cyclelength-treatment']

In [3]:
import pandas as pd
alldf = pd.read_csv("../data/raw_aac.csv")
print (alldf.shape)
print (alldf.columns)
alldf.head()

(45, 12)
Index(['design_group_id', 'nct_id', 'group_type', 'title', 'arm_desc',
       'intervention_id', 'intervention_type', 'name', 'intervention_desc',
       'arm_title_desc', 'intervention_name_desc',
       'intervention_type_name_desc'],
      dtype='object')


,design_group_id,nct_id,group_type,title,arm_desc,intervention_id,intervention_type,name,intervention_desc,arm_title_desc,intervention_name_desc,intervention_type_name_desc
0,13299803,NCT00000435,Placebo Comparator,A,Subjects randomized to arm A received 25mg/day...,12910066,Drug,None-placebo,placebo was taken in pill form at 25mg/day for...,A\n\nSubjects randomized to arm A received 25m...,None-placebo: placebo was taken in pill form a...,Drug\nNone-placebo: placebo was taken in pill ...
1,13299804,NCT00000435,Active Comparator,B,Subjects randomized to Arm B received 25mg/day...,12910065,Drug,dnaJ peptide,dnaJP1 was taken in pill form at 25mg/day for ...,B\n\nSubjects randomized to Arm B received 25m...,dnaJ peptide: dnaJP1 was taken in pill form at...,Drug\ndnaJ peptide: dnaJP1 was taken in pill f...
2,13690845,NCT00001337,Experimental,Arm A,EPOCH + Rituximab every 3 weeks for 6 cycles.,13274570,Drug,EPOCH,Combination chemotherapy given with Rituximab ...,Arm A\n\nEPOCH + Rituximab every 3 weeks for 6...,EPOCH: Combination chemotherapy given with Rit...,Drug\nEPOCH: Combination chemotherapy given wi...
3,13690845,NCT00001337,Experimental,Arm A,EPOCH + Rituximab every 3 weeks for 6 cycles.,13274571,Biological,Rituximab,Rituximab given on Day 1 of combination chemot...,Arm A\n\nEPOCH + Rituximab every 3 weeks for 6...,Rituximab: Rituximab given on Day 1 of combina...,Biological\nRituximab: Rituximab given on Day ...
4,13732309,NCT00001563,Experimental,1,EPOCH-R every 3 weeks for up to 6 cycle,13311178,Biological,Filgrastim,Filgrastim after EPOCH-R from Day 6 for 10 day...,1\n\nEPOCH-R every 3 weeks for up to 6 cycle,Filgrastim: Filgrastim after EPOCH-R from Day ...,Biological\nFilgrastim: Filgrastim after EPOCH...


In [4]:
name_sep = ": "
sent_sep = "."
tit_sep = "\n\n"

# Join title and arm description
alldf["arm_title_desc"] = alldf["title"]+tit_sep+alldf["arm_desc"]
# Join name and intervention desc
alldf["intervention_name_desc"] = alldf["name"]+name_sep+alldf["intervention_desc"]+sent_sep
# Join intervention type and name and intervention desc
#alldf["intervention_type_name_desc"] = alldf["intervention_type"]+tit_sep+alldf["intervention_name_desc"]
alldf.head()

,design_group_id,nct_id,group_type,title,arm_desc,intervention_id,intervention_type,name,intervention_desc,arm_title_desc,intervention_name_desc,intervention_type_name_desc
0,13299803,NCT00000435,Placebo Comparator,A,Subjects randomized to arm A received 25mg/day...,12910066,Drug,None-placebo,placebo was taken in pill form at 25mg/day for...,A\n\nSubjects randomized to arm A received 25m...,None-placebo: placebo was taken in pill form a...,Drug\nNone-placebo: placebo was taken in pill ...
1,13299804,NCT00000435,Active Comparator,B,Subjects randomized to Arm B received 25mg/day...,12910065,Drug,dnaJ peptide,dnaJP1 was taken in pill form at 25mg/day for ...,B\n\nSubjects randomized to Arm B received 25m...,dnaJ peptide: dnaJP1 was taken in pill form at...,Drug\ndnaJ peptide: dnaJP1 was taken in pill f...
2,13690845,NCT00001337,Experimental,Arm A,EPOCH + Rituximab every 3 weeks for 6 cycles.,13274570,Drug,EPOCH,Combination chemotherapy given with Rituximab ...,Arm A\n\nEPOCH + Rituximab every 3 weeks for 6...,EPOCH: Combination chemotherapy given with Rit...,Drug\nEPOCH: Combination chemotherapy given wi...
3,13690845,NCT00001337,Experimental,Arm A,EPOCH + Rituximab every 3 weeks for 6 cycles.,13274571,Biological,Rituximab,Rituximab given on Day 1 of combination chemot...,Arm A\n\nEPOCH + Rituximab every 3 weeks for 6...,Rituximab: Rituximab given on Day 1 of combina...,Biological\nRituximab: Rituximab given on Day ...
4,13732309,NCT00001563,Experimental,1,EPOCH-R every 3 weeks for up to 6 cycle,13311178,Biological,Filgrastim,Filgrastim after EPOCH-R from Day 6 for 10 day...,1\n\nEPOCH-R every 3 weeks for up to 6 cycle,Filgrastim: Filgrastim after EPOCH-R from Day ...,Biological\nFilgrastim: Filgrastim after EPOCH...


In [5]:
# two tables
# arm table name/desc
# intervention name/desc

# groupby design_group_id & intervention_id so we can have multiple interventions in a single arm

# merge arm & intervention results

# intervention desc would be first priority
# if not found, check in arm - drug name

# [
#     {
#         "ARM_NAME (group_type + title,)": { 'Treatment A (name)' : } 
    
        
#     }
# ]

# {
#     "Placebo Comparator A" : [
#         { "None-placebo (RxNorm Code / Generic Name) Asipirin (Drug)" : {"Dosage" : "10" , "Strength": "xx", "Route" : "oral", "Disposition", "Frequency"} },
#         { "None-placebo (RxNorm Code / Generic Name) Lipitor (Biological)" : {"Dosage" : "10" , "Strength": "xx", "Route" : "oral", "Disposition", "Frequency"} },
        
#          ]
# }
    

In [6]:
from ctdi_treatment.treatment_pipeline import build_treatment_pipeline, build_df, LightPipeline, PipelineModel
#build_or_load = "build"#"load"
#s = time.time()
#if build_or_load=="build":
pipeline = build_treatment_pipeline()
p_model = pipeline.fit(spark.createDataFrame([("",)]).toDF("text"))
l_model = LightPipeline(p_model)
# else:
#     plm = PipelineModel.load(f"models/{pl_name}")
# print(time.time()-s)

embeddings_clinical download started this may take some time.
Approximate size to download 1.6 GB
[OK!]
pos_clinical download started this may take some time.
Approximate size to download 1.5 MB
[OK!]
dependency_conllu download started this may take some time.
Approximate size to download 16.7 MB
[OK!]
ner_posology download started this may take some time.
Approximate size to download 13.8 MB
[OK!]
ner_clinical download started this may take some time.
Approximate size to download 13.9 MB
[OK!]
ner_clinical_large download started this may take some time.
Approximate size to download 13.9 MB
[OK!]
ner_jsl download started this may take some time.
Approximate size to download 14.5 MB
[OK!]
sbiobert_base_cased_mli download started this may take some time.
Approximate size to download 384.3 MB
[OK!]
sbiobertresolve_rxnorm_disposition download started this may take some time.
Approximate size to download 804.2 MB
[OK!]


In [11]:
annotation_df = alldf.groupby(["nct_id","design_group_id","group_type","title"]).agg(
    intervention_name_desc=("intervention_name_desc", tit_sep.join),
    arm_title_desc = ('arm_title_desc', tit_sep.join),
    intervention_type = ( 'intervention_type', '|'.join),
#     group_type = ( 'group_type' , tit_sep.join ),
#     title = (  'title', tit_sep.join )
).reset_index()
annotation_df.head(10)

,nct_id,design_group_id,group_type,title,intervention_name_desc,arm_title_desc,intervention_type
0,NCT00000435,13299803,Placebo Comparator,A,None-placebo: placebo was taken in pill form a...,A\n\nSubjects randomized to arm A received 25m...,Drug
1,NCT00000435,13299804,Active Comparator,B,dnaJ peptide: dnaJP1 was taken in pill form at...,B\n\nSubjects randomized to Arm B received 25m...,Drug
2,NCT00001337,13690845,Experimental,Arm A,EPOCH: Combination chemotherapy given with Rit...,Arm A\n\nEPOCH + Rituximab every 3 weeks for 6...,Drug|Biological
3,NCT00001563,13732309,Experimental,1,Filgrastim: Filgrastim after EPOCH-R from Day ...,1\n\nEPOCH-R every 3 weeks for up to 6 cycle\n...,Biological|Biological|Drug
4,NCT00001575,13299883,Experimental,Anti-Tac yttrium 90-labeled humanized anti-Tac...,Y-90 Humanized Anti-Tac: 10 mCi (if a bone mar...,Anti-Tac yttrium 90-labeled humanized anti-Tac...,Biological|Drug
5,NCT00001586,13299880,Experimental,Low-Intermediate Risk B-Cell Pts,Leukemic or stroma cells: Patients are eligibl...,Low-Intermediate Risk B-Cell Pts\n\nPreviously...,Other
6,NCT00001586,13299881,Experimental,Intermediate-high Risk B-Cell Pts,Rituximab: Rituxan.\n\nFludarabine phosphate: ...,Intermediate-high Risk B-Cell Pts\n\nPreviousl...,Biological|Drug
7,NCT00001984,13299801,Experimental,Alemtuzumab and DSG,Alemtuzumab and DSG: Alemtuzumab was administe...,Alemtuzumab and DSG\n\nThe recipients of live ...,Drug
8,NCT00002548,13296605,Active Comparator,HDCTX and PBSC,doxorubicin hydrochloride: 10 mg/m2/day contin...,HDCTX and PBSC\n\nHigh dose chemotherapy with ...,Drug|Drug|Drug|Drug|Procedure
9,NCT00002548,13296606,Experimental,HDCTX with PBSC and Autologous BMT,carmustine: 20 mg/m2 I.V. day 1 q 35 days.\n\n...,HDCTX with PBSC and Autologous BMT\n\nHigh dos...,Drug|Drug|Drug|Drug|Drug|Drug|Procedure|Proced...


In [20]:
intervention_res = l_model.fullAnnotate(annotation_df['intervention_name_desc'].values.tolist())
arm_res = l_model.fullAnnotate(annotation_df['arm_title_desc'].values.tolist())

In [21]:
intervention_res[0]['resolution_rxnorm']

[Annotation(entity, 0, 11, 1872077, {'all_k_results': '1872077:::220959:::1314253', 'all_k_distances': '7.4137:::8.0947:::8.1481', 'confidence': '0.5035', 'all_k_cosine_distances': '0.0806:::0.0967:::0.0960', 'all_k_resolutions': 'neverpain:::xs no aspirin pr:::nonanal', 'target_text': 'None-placebo', 'all_k_aux_labels': '-:::-:::-', 'token': 'None-placebo', 'resolved_text': 'neverpain', 'all_k_confidences': '0.5035:::0.2549:::0.2416', 'distance': '7.4137', 'sentence': '0'}),
 Annotation(entity, 14, 20, 8375, {'all_k_results': '8375:::1164457:::1164456', 'all_k_distances': '0.0000:::6.8203:::7.0958', 'confidence': '0.9981', 'all_k_cosine_distances': '0.0000:::0.0718:::0.0775', 'all_k_resolutions': 'placebo:::placebo pill:::placebo oral product', 'target_text': 'placebo', 'all_k_aux_labels': '-:::-:::-', 'token': 'placebo', 'resolved_text': 'placebo', 'all_k_confidences': '0.9981:::0.0011:::0.0008', 'distance': '0.0000', 'sentence': '1'}),
 Annotation(entity, 35, 38, 1151133, {'all_k_re

In [15]:
# print (intervention_res[0]['all_chunk'])
# print (intervention_res[0]['resolution_rxnorm'])
for i, j in zip (intervention_res[0]['resolution_rxnorm'], intervention_res[0]['all_chunk']):
    print (j.result)
    print (i.metadata['resolved_text'])

None-placebo
neverpain
placebo
placebo
pill
pill
25mg/day
perazine 25 mg
for 6 months
sulfur 6 x


In [30]:
id_ = -2
from sparknlp_display import EntityResolverVisualizer, RelationExtractionVisualizer
EntityResolverVisualizer().display(intervention_res[id_], 'drug_chunk', 'resolution_rxnorm' )
RelationExtractionVisualizer().display(intervention_res[id_], 'relations' )

EntityResolverVisualizer().display(arm_res[id_], 'full_chunk', 'resolution_rxnorm' )
RelationExtractionVisualizer().display(arm_res[id_], 'relations' )

In [23]:
## raw_sentence
## raw_drug
## if drug found , don't look for treatment

## New code - run this now

In [29]:
# two tables
# arm table name/desc
# intervention name/desc

# groupby design_group_id & intervention_id so we can have multiple interventions in a single arm

# merge arm & intervention results

# intervention desc would be first priority
# if not found, check in arm - drug name

def find_formula(k, l, mode='small', respect_section=None):
        
    try:
        lrgst = min(filter(lambda x: x > k, l))
    except:
        lrgst = None
    try:
        clst = min(list(filter(lambda x: x != k, l)), key=lambda x: abs(x-k))
    except:
        clst = None
    try:
        smlst = max(filter(lambda x: x < k, l))
    except:
        smlst = None
    
    return smlst, clst, lrgst


def relate_drugs(res, sentences, relations, resolution_res, intervention_type):
    
    drug_ent_list = ['drug', 'treatment']
    
    resolutions_list = {}
    for i in resolution_res:
        try:
            disp = i.metadata['all_k_aux_labels'].split(':::')[0]
        except:
            disp = ''
        if disp == '-': disp = ''
        resolutions_list[int(i.begin)] = { 'resolved_text' : i.metadata['resolved_text'],
                                          'rxnorm_code': i.result,
                                     'disposition' : disp}
        
    sentences_list = {}
    for i in sentences:
        sentences_list[int(i.metadata['sentence'])] = i.result
    
    #print (sentences_list)
    
    drugs_list = {}
    other_entities = {}
    
    drug_in_this = False
    for i in res:
        if i.metadata['entity'].lower().strip() == 'drug':
            drug_in_this = True
    
    if drug_in_this == True:
        drug_ent_list = ['drug']
        
    for i in res:
        #print(int(i.begin))
        if i.metadata['entity'].lower().strip() in drug_ent_list:
            drugs_list[int(i.begin)] = { resolutions_list[int(i.begin)]['resolved_text'] : {
                                            "raw_sentence": sentences_list[int(i.metadata['sentence'])],
                                            "raw_drug" : i.result,
                                            "drug_entity": i.metadata['entity'],
                                            "resolved_drug": resolutions_list[int(i.begin)]['resolved_text'],
                                            'intervention_type': intervention_type,
                                            'rxnorm_code': resolutions_list[int(i.begin)]['rxnorm_code'],
                                            "associated_details" : { 'dosage' : '', 'form': '', 'route': '',
                                                                       'strength': '', 'frequency': '', 
                                                                        'duration': '', 
                                                'disposition': resolutions_list[int(i.begin)]['disposition'],
                                                                        'relativedate' : '', 
                                                                         'administration' : '', 'cyclelength' : '',
                                                                       }
                                           }
                                       }
        elif i.metadata['entity'].lower().strip() != 'treatment':
            other_entities[int(i.begin)] = {'type': i.metadata['entity'].strip().lower(), 'chunk': i.result}
    
    done_other_entities = []
    for rel in relations:
        if rel.metadata['entity1'].lower().strip() in drug_ent_list:
            drug_ent_id = int(rel.metadata['entity1_begin'])
            oth_ent = other_entities[int(rel.metadata['entity2_begin'])]
            drugs_list[drug_ent_id][resolutions_list[drug_ent_id]['resolved_text']]['associated_details'][oth_ent['type']] = oth_ent['chunk']
            
            done_other_entities.append(int(rel.metadata['entity2_begin']))
            
        elif rel.metadata['entity2'].lower().strip() in drug_ent_list:
            drug_ent_id = int(rel.metadata['entity2_begin'])
            oth_ent = other_entities[int(rel.metadata['entity1_begin'])]
            drugs_list[drug_ent_id][resolutions_list[drug_ent_id]['resolved_text']]['associated_details'][oth_ent['type']] = oth_ent['chunk']
        
            done_other_entities.append(int(rel.metadata['entity1_begin']))
    
    remaining_entities = set(list(other_entities.keys())) - set(done_other_entities)
    
    for rem_ent in remaining_entities:
        oth_ent = other_entities[rem_ent]
        smlst, clst, lrgst = find_formula(rem_ent, list(drugs_list.keys()))
        #this_sent = int(i.metadata['sentence'])
        if smlst != None:
            drugs_list[smlst][resolutions_list[smlst]['resolved_text']]['associated_details'][oth_ent['type']] = oth_ent['chunk']
        elif lrgst != None:# and drugs_sents[lrgst] == this_sent:
            drugs_list[lrgst][resolutions_list[lrgst]['resolved_text']]['associated_details'][oth_ent['type']] = oth_ent['chunk']
        
        
    return list(drugs_list.values())
                             

assert len(intervention_res) == annotation_df.shape[0] == len(arm_res)
all_res = []
for index in range(annotation_df.shape[0]):
    json_obj = {}
    
    ires_rec = intervention_res[index]
    ares_rec = arm_res[index]
    row_rec = annotation_df.iloc[index]
    
    found_drugs = False
    found_drugs_i = relate_drugs(ires_rec['full_chunk'], ires_rec['sentence'],
                                 ires_rec['relations'], ires_rec['resolution_rxnorm'] , 
                                 row_rec['intervention_type']) 
    found_drugs_a = relate_drugs(ares_rec['full_chunk'], ares_rec['sentence'],
                                 ares_rec['relations'], ares_rec['resolution_rxnorm'] , None)
    if found_drugs_i:
        json_obj[row_rec['group_type'] + ' ' + row_rec['title']] = found_drugs_i
    elif found_drugs_a:
        json_obj[row_rec['group_type'] + ' ' + row_rec['title']] = found_drugs_a
    
    all_res.append(json_obj)
    
# [
#     {
#         "ARM_NAME (group_type + title,)": { 'Treatment A (name)' : } 
    
        
#     }
# ]

# {
#     "Placebo Comparator A" : [
#         { "None-placebo (RxNorm Code / Generic Name) Asipirin (Drug)" : {"Dosage" : "10" , "Strength": "xx", "Route" : "oral", "Disposition", "Frequency"} },
#         { "None-placebo (RxNorm Code / Generic Name) Lipitor (Biological)" : {"Dosage" : "10" , "Strength": "xx", "Route" : "oral", "Disposition", "Frequency"} },
        
#          ]
# }

all_res
    
    

[{'Placebo Comparator A': [{'neverpain': {'raw_sentence': 'None-placebo',
     'raw_drug': 'None-placebo',
     'drug_entity': 'Drug',
     'resolved_drug': 'neverpain',
     'intervention_type': 'Drug',
     'rxnorm_code': '1872077',
     'associated_details': {'dosage': '',
      'form': '',
      'route': '',
      'strength': '',
      'frequency': '',
      'duration': '',
      'disposition': '',
      'relativedate': '',
      'administration': '',
      'cyclelength': ''}}},
   {'placebo': {'raw_sentence': 'placebo was taken in pill form at 25mg/day for 6 months.',
     'raw_drug': 'placebo',
     'drug_entity': 'Drug',
     'resolved_drug': 'placebo',
     'intervention_type': 'Drug',
     'rxnorm_code': '8375',
     'associated_details': {'dosage': '',
      'form': 'pill',
      'route': '',
      'strength': '25mg/day',
      'frequency': '',
      'duration': '',
      'disposition': '',
      'relativedate': '',
      'administration': '',
      'cyclelength': 'for 6 mont

## Old Code

In [52]:
##### Old code


# two tables
# arm table name/desc
# intervention name/desc

# groupby design_group_id & intervention_id so we can have multiple interventions in a single arm

# merge arm & intervention results

# intervention desc would be first priority
# if not found, check in arm - drug name

def find_drugs(res, resolution_res):
    
    resolutions = {}
    for i in resolution_res:
        resolutions[int(i.begin)] = i.metadata['resolved_text']
    
    drugs = []
    other_entities = []
    drugs = {}
    drugs_sents = {}
    for i in res:
        if i.metadata['entity'].lower().strip() == 'drug':
            #drugs[int(i.begin)] = { 'entity': 'drug', 'result': i.result, 'sentence': i.metadata['sentence'] }
            try:
                drugs[int(i.begin)] = {resolutions[int(i.begin)] : { 'Dosage' : '', 'Form': '', 'Route': '',
                                                                   'Strength': '', 'Frequency': '', 
                                                                    'Duration': '', 'Disposition': ''}}
                drugs_sents[int(i.begin)] = int(i.metadata['sentence'])
            except:
                pass
    for i in res:
        
        if i.metadata['entity'].lower().strip() in ['strength', 'route', 'frequency', 'form', 'dosage','duration']:
            smlst, clst, lrgst = find_formula(int(i.begin), list(drugs.keys()))
            this_sent = int(i.metadata['sentence'])
            if smlst != None:
                drugs[smlst][resolutions[smlst]][i.metadata['entity'].strip().title()] = i.result
            if lrgst != None and drugs_sents[lrgst] == this_sent:
                drugs[lrgst][resolutions[lrgst]][i.metadata['entity'].strip().title()] = i.result
            #elif smlst != None and drugs_sents[smlst] == this_sent:
                
    return list(drugs.values())
                             

assert len(intervention_res) == annotation_df.shape[0] == len(arm_res)
all_res = []
for index in range(annotation_df.shape[0]):
    json_obj = {}
    
    ires_rec = intervention_res[index]
    ares_rec = arm_res[index]
    row_rec = annotation_df.iloc[index]
    
    found_drugs = False
    found_drugs_i = find_drugs(ires_rec['all_chunk'], ires_rec['resolution_rxnorm'] ) 
    found_drugs_a = find_drugs(ares_rec['all_chunk'], ares_rec['resolution_rxnorm'])
    if found_drugs_i:
        json_obj[row_rec['group_type'] + ' ' + row_rec['title']] = found_drugs_i
    elif found_drugs_a:
        json_obj[row_rec['group_type'] + ' ' + row_rec['title']] = found_drugs_a
    
    all_res.append(json_obj)
    
# [
#     {
#         "ARM_NAME (group_type + title,)": { 'Treatment A (name)' : } 
    
        
#     }
# ]

# {
#     "Placebo Comparator A" : [
#         { "None-placebo (RxNorm Code / Generic Name) Asipirin (Drug)" : {"Dosage" : "10" , "Strength": "xx", "Route" : "oral", "Disposition", "Frequency"} },
#         { "None-placebo (RxNorm Code / Generic Name) Lipitor (Biological)" : {"Dosage" : "10" , "Strength": "xx", "Route" : "oral", "Disposition", "Frequency"} },
        
#          ]
# }

all_res
    
    

[{'Placebo Comparator A': [{'neverpain': {'Dosage': '',
     'Form': '',
     'Route': '',
     'Strength': '',
     'Frequency': '',
     'Duration': '',
     'Disposition': ''}},
   {'placebo': {'Dosage': '',
     'Form': 'pill',
     'Route': '',
     'Strength': '25mg/day',
     'Frequency': '',
     'Duration': 'for 6 months',
     'Disposition': ''}}]},
 {'Active Comparator B': [{'n-acetyl dipeptide-1': {'Dosage': '',
     'Form': '',
     'Route': 'po',
     'Strength': '25mg/day',
     'Frequency': '',
     'Duration': '',
     'Disposition': ''}}]},
 {'Experimental Arm A': [{'rituximab': {'Dosage': '',
     'Form': '',
     'Route': '',
     'Strength': '',
     'Frequency': '',
     'Duration': '',
     'Disposition': ''}},
   {'relacon-hc': {'Dosage': '',
     'Form': '',
     'Route': 'IV',
     'Strength': '',
     'Frequency': 'every 3 weeks',
     'Duration': 'for 6 cycles',
     'Disposition': ''}},
   {'rituximab': {'Dosage': '',
     'Form': '',
     'Route': '',
    

In [16]:
import json
with open('output_new.json', 'w') as f_:
    f_.write(json.dumps(all_res, indent=4))